In [70]:
import shutil

import aseg_gdf2

Use the third-party package ``fortranformat`` to parse the GDF2 format fields

In [47]:
import fortranformat as ff

In [5]:
gdf = aseg_gdf2.read(r'c:\users\kinve\code\aseg_gdf2\tests\example_datasets\3bcfc711\GA1286_Waveforms')

In [35]:
df = gdf.df()
df.head()

,FLTNUM,Rx_Voltage,Flight,Time,Tx_Current
0,1.0,-0.0,1,0.0052,0.00176
1,1.0,-0.0,1,0.0104,0.00176
2,1.0,-0.0,1,0.0156,0.00176
3,1.0,-0.0,1,0.0208,0.00176
4,1.0,-0.0,1,0.0260,0.00176


In [36]:
df["FLTNUM"] = df.FLTNUM * 1000

In [37]:
df

,FLTNUM,Rx_Voltage,Flight,Time,Tx_Current
0,1000.0,-0.0,1,0.0052,0.00176
1,1000.0,-0.0,1,0.0104,0.00176
2,1000.0,-0.0,1,0.0156,0.00176
3,1000.0,-0.0,1,0.0208,0.00176
4,1000.0,-0.0,1,0.0260,0.00176
...,...,...,...,...,...
23035,2000.0,-0.0,2,59.9740,-0.00170
23036,2000.0,-0.0,2,59.9792,-0.00170
23037,2000.0,-0.0,2,59.9844,-0.00170
23038,2000.0,-0.0,2,59.9896,-0.00170


In [38]:
gdf.record_types

{'': {'fields': [{'name': 'FLTNUM',
    'format': 'F10.1',
    'unit': '',
    'null': None,
    'long_name': 'FlightNumber',
    'comment': '',
    'cols': 1,
    'width': 10},
   {'name': 'Rx_Voltage',
    'format': 'F10.5',
    'unit': 'Volt',
    'null': '-99.99999',
    'long_name': 'Rx_Voltage',
    'comment': '',
    'cols': 1,
    'width': 10},
   {'name': 'Flight',
    'format': 'I6',
    'unit': '',
    'null': '-9999',
    'long_name': 'Flight',
    'comment': '',
    'cols': 1,
    'width': 6},
   {'name': 'Time',
    'format': 'F10.4',
    'unit': 'msec',
    'null': '-999.9999',
    'long_name': 'Time',
    'comment': '',
    'cols': 1,
    'width': 10},
   {'name': 'Tx_Current',
    'format': 'F13.5',
    'unit': 'Amp',
    'null': '-99999.99999',
    'long_name': 'Tx_Current',
    'comment': '',
    'cols': 1,
    'width': 13}],
  'format': None}}

Create new DFN and DAT files

In [44]:
gdf.dfn_filename

'c:\\users\\kinve\\code\\aseg_gdf2\\tests\\example_datasets\\3bcfc711\\GA1286_Waveforms.DFN'

In [45]:
gdf.dat_filename

'c:\\users\\kinve\\code\\aseg_gdf2\\tests\\example_datasets\\3bcfc711\\GA1286_Waveforms.dat'

In [49]:
new_dfn_filename = gdf.dfn_filename.lower().replace(".dfn", "_v2.dfn")
new_dat_filename = gdf.dat_filename.lower().replace(".dat", "_v2.dat")

Duplicate the dfn file.

In [50]:
shutil.copyfile(gdf.dfn_filename, new_dfn_filename)

'c:\\users\\kinve\\code\\aseg_gdf2\\tests\\example_datasets\\3bcfc711\\ga1286_waveforms_v2.dfn'

Obtain formatting fields for a given record type (this only works in the simple situation when there is one record type).

Also get the values to fill per column for NaN values.

In [68]:
rt = "" # NULL record type
format_fields = ", ".join([field["format"] for field in gdf.record_types[rt]["fields"]])
null_values = {
    field["name"]: float(field["null"]) if field["null"] else None
    for field in gdf.record_types[rt]["fields"]
}

writer = ff.FortranRecordWriter("(" + format_fields + ")")

In [69]:
with open(new_dat_filename, "w") as new_dat:
    for idx, series in df.iterrows():
        series_filled = series.fillna(null_values)
        line = writer.write(series_filled).replace("\n", "")
        new_dat.write(line + "\n")

Check the output file.

In [71]:
gdf_v2 = aseg_gdf2.read(new_dfn_filename)

In [72]:
gdf_v2.df()

,FLTNUM,Rx_Voltage,Flight,Time,Tx_Current
0,1000.0,0.0,1,0.0052,0.00176
1,1000.0,0.0,1,0.0104,0.00176
2,1000.0,0.0,1,0.0156,0.00176
3,1000.0,0.0,1,0.0208,0.00176
4,1000.0,0.0,1,0.0260,0.00176
...,...,...,...,...,...
23035,2000.0,0.0,2,59.9740,-0.00170
23036,2000.0,0.0,2,59.9792,-0.00170
23037,2000.0,0.0,2,59.9844,-0.00170
23038,2000.0,0.0,2,59.9896,-0.00170


Compare old to new file contents.

In [76]:
print("Original file")
with open(gdf.dat_filename, "r") as new_dat_file:
    for i, line in enumerate(new_dat_file):
        print(line.strip("\n"))
        if i == 10:
            break

Original file
       1.0  -0.00000     1    0.0052      0.00176
       1.0  -0.00000     1    0.0104      0.00176
       1.0  -0.00000     1    0.0156      0.00176
       1.0  -0.00000     1    0.0208      0.00176
       1.0  -0.00000     1    0.0260      0.00176
       1.0  -0.00000     1    0.0312      0.00176
       1.0  -0.00000     1    0.0365      0.00176
       1.0  -0.00000     1    0.0417      0.00176
       1.0  -0.00000     1    0.0469      0.00176
       1.0  -0.00000     1    0.0521      0.00176
       1.0  -0.00000     1    0.0573      0.00176


In [77]:
print ("New file")
with open(new_dat_filename, "r") as new_dat_file:
    for i, line in enumerate(new_dat_file):
        print(line.strip("\n"))
        if i == 10:
            break

New file
    1000.0   0.00000     1    0.0052      0.00176
    1000.0   0.00000     1    0.0104      0.00176
    1000.0   0.00000     1    0.0156      0.00176
    1000.0   0.00000     1    0.0208      0.00176
    1000.0   0.00000     1    0.0260      0.00176
    1000.0   0.00000     1    0.0312      0.00176
    1000.0   0.00000     1    0.0365      0.00176
    1000.0   0.00000     1    0.0417      0.00176
    1000.0   0.00000     1    0.0469      0.00176
    1000.0   0.00000     1    0.0521      0.00176
    1000.0   0.00000     1    0.0573      0.00176


```
Old file:
       1.0  -0.00000     1    0.0573      0.00176
New file:
    1000.0   0.00000     1    0.0052      0.00176
```